In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

# MNIST 학습데이터 준비

In [ ]:
# MNIST
import tensorflow.keras.datasets as ds

(x_train,y_train),(x_test,y_test) = ds.mnist.load_data()

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
# 학습 데이터 정규화
x_trainF = x_train.reshape(60000,784)   # 1차원 구조로 변경
x_testF = x_test.reshape(10000,784)

x_trainF = x_trainF.astype('float32')   # 0~1로 정규화하기 위해 실수형으로 변환
x_testF = x_testF.astype('float32')

x_trainF /= 255.0                       # 0~1로 정규화
x_testF /= 255.0

x_trainF.shape, x_testF.shape

In [ ]:
y_train10 = tf.keras.utils.to_categorical(y_train,10)   # one-hot 인코딩
y_test10 = tf.keras.utils.to_categorical(y_test,10)

y_train10.shape, y_test10.shape

# 인공신경망(Artificial Neural Network, ANN)

### 신경망 학습 : 예, MLP

In [ ]:
# 학습 모델 : 신경망
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

mlp_model = Sequential()
mlp_model.add(Input(shape=(784,)))
mlp_model.add(Dense(128, activation=tf.nn.tanh))# Input layer
mlp_model.add(Dense(10, activation=tf.nn.softmax)) # adding an output layer (with 10 possible outputs for the 10 digits we need to predict)

In [ ]:
# 학습
from tensorflow.keras.optimizers import SGD

mlp_model.compile(loss='MSE', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])
history = mlp_model.fit(x_trainF, y_train10, batch_size=128, epochs=50, validation_data=(x_testF,y_test10), verbose=2)

In [ ]:
# plot loss and acc
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('Loss value')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch')
plt.ylabel('acc value')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# 평가 : 학습이 끝난 후 테스트 데이터로 모델의 정확도 측정
mlp_acc = mlp_model.evaluate(x_testF, y_test10, verbose=0)
print('정확도 = ', mlp_acc[1]*100)

In [ ]:
# 예측 : 학습 모델에 입력데이터에 대한 출력 예측
output = mlp_model.predict(x_testF[0].reshape(1, 784))
print(output)
top1 = output.argmax(axis=1)   # 가장 큰 확률의 클래스(라벨)
print(top1)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[i], cmap=plt.cm.binary)
    plt.xlabel(str(y_test[i]) + '-->' + str(mlp_model.predict(x_testF[i].reshape(1, 784)).argmax(axis=1)))
plt.show()

# 딥러닝 : CNN

## LeNet5

### CNN 학습을 위한 학습데이터

In [ ]:
# 2차원 구조(vs. mlp)
x_trainC = x_train.astype(np.float32)/255.0  # 0~1로 정규화
x_testC = x_test.astype(np.float32)/255.0

x_trainC.shape, x_testC.shape

In [ ]:
y_train10 = tf.keras.utils.to_categorical(y_train,10)   # one-hot 인코딩
y_test10 = tf.keras.utils.to_categorical(y_test,10)

y_train10.shape, y_test10.shape

### CNN 학습

In [ ]:
# 학습 모델 : LeNet5
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
from keras.optimizers import Adam

cnn = Sequential()
cnn.add(Conv2D(6,(5,5),padding='same',activation='relu',input_shape=(28,28,1)))
cnn.add(MaxPooling2D(pool_size=(2,2),strides=2))
cnn.add(Conv2D(16,(5,5),padding='valid',activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2),strides=2))
cnn.add(Flatten())
cnn.add(Dense(units=120,activation='relu'))
cnn.add(Dense(units=84,activation='relu'))
cnn.add(Dense(units=10,activation='softmax'))

In [ ]:
cnn.summary()

In [ ]:
# 학습
cnn.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
cnn.fit(x_trainC, y_train10, batch_size=128, epochs=50, validation_data=(x_testC,y_test10), verbose=2)

In [ ]:
# 평가
cnn_acc = cnn.evaluate(x_testC,y_test10, verbose=0)
print('정확률=',cnn_acc[1]*100)

In [ ]:
# 예측
output = cnn.predict(x_testC[0].reshape(1, 28,28,1))

top1 = output.argmax(axis=1)   # 가장 큰 확률의 클래스(라벨)
print(output)
print(top1)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_test[i], cmap=plt.cm.binary)
    plt.xlabel(str(y_test[i]) + '-->' + str(cnn.predict(x_testC[i].reshape(1,28,28,1)).argmax(axis=1)))
plt.show()

In [ ]:
cnn.save('lenet_trained.keras') #.h5')

### 가중치 시각화

In [ ]:
# 세번째 layer의 가중치 불러오기
filters , bias = cnn.layers[2].get_weights()
print(filters.shape)

# 0-1 정규화
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

f = filters[:,:,:,0]
print(f[:,:,0])
fig = plt.figure(figsize=(2,2))
plt.imshow(f[:,:,0], cmap='gray')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
n_filters, ix = 4, 1
fig = plt.figure(figsize=(8,8))
for i in range(n_filters):
    f = filters[:,:,:,i]
    for j in range(4):
        plt.subplot(n_filters,4,ix)
        plt.imshow(f[:,:,j], cmap='gray')
        ix+=1
plt.show()